<a href="https://colab.research.google.com/github/Fazira2025/weatherwise-template/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---




In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai


## 📦 Setup and Configuration
Import required packages and setup environment.

In [3]:
import requests

## 🌤️ Weather Data Functions

In [4]:
# Define get_weather_data() function here
def get_weather_data(location):
    try:
        url = f'https://wttr.in/{location}'
        response = requests.get(url)
        response.raise_for_status()
        print(response.text)
    except requests.exceptions.HTTPError as errh:
        print(f'HTTP Error: {errh}')
    except requests.exceptions.ConnectionError as errc:
        print(f'Connection Error: {errc}')
    except requests.exceptions.Timeout as errt:
        print(f'Timeout Error: {errt}')
    except requests.exceptions.RequestException as err:
        print(f'Error: {err}')

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here


In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here


## 🧭 User Interface

In [5]:
# Define menu functions using pyinputplus or ipywidgets here
def main():
  name = input('🤖: Hi! Please enter your name: ')
  location = input(f'🤖: Which location would you like to check, {name}? ')
  while True:
        print('\nHello! Welcome to Weather Wise. How can I help you?😊')
        print("-" * 53)
        print('Please enter your choice (1 - 4):')
        print(f'1. Check Current Weather & View Weather Forecast at {location}')
        print(f'2. Activities Suggestions at {location} (Today, Tomorrow, Day After Tomorrow)')
        print('3. Check Other Location ')
        print('4. Exit Program ')

        choice = input('Enter your choice (1-4): ')

        if choice == "1":
           get_weather_data(location)
        elif choice == "2":
            print('You chose Choice 2')
        elif choice == "3":
            print('')
            location = input(f'{name}, Please enter your new location: ')
            print(f'🤖: Your new location is {location}')
            print('You chose Choice 3')
        elif choice == "4":
            print(f'Thankyou! Have a nice day {name} 🌸')
            break
        else:
            print("\033[1;31mInvalid choice. Please type 1-4 as a number!\033[0m") #error_handling

## 🧩 Main Application Logic

In [6]:
# Tie everything together here
if __name__ == "__main__":
    title = "Weather Wise"
    blue_bold = "\033[1;94m"  # 1 for bold, 94 for light blue
    color = "\033[0m"

    print("=" * 53)
    print("" + blue_bold + title.center(51) + color + "")
    print("=" * 53)

    main()

                    Weather Wise                   
🤖: Hi! Please enter your name: Zizi
🤖: Which location would you like to check, Zizi? Perth

Hello! Welcome to Weather Wise. How can I help you?😊
-----------------------------------------------------
Please enter your choice (1 - 4):
1. Check Current Weather & View Weather Forecast at Perth
2. Activities Suggestions at Perth (Today, Tomorrow, Day After Tomorrow)
3. Check Other Location 
4. Exit Program 
Enter your choice (1-4): 1
Weather report: Perth

      \   /     Clear
       .-.      +55(53) °F     
    ― (   ) ―   ↖ 11 mph       
       `-’      6 mi           
      /   \     0.0 in         
                                                       ┌─────────────┐                                                       
┌──────────────────────────────┬───────────────────────┤  Mon 19 May ├───────────────────────┬──────────────────────────────┐
│            Morning           │             Noon      └──────┬──────┘     Evening        

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.